# Certificados y PKI

Vamos a crear una Public Key Infrastructure desde línea de comandos. Para este ejercicio necesitas tener acceso a una máquina con OpenSSL. Algunas posibilidades:

- OpenSSL puede instalarse en Windows como aplicación independiente, y se ejecuta desde línea de comandos
- OpenSSL ya viene instalada en Linux y OSX
- Puede instalarse también en máquinas virtuales, a través de WSL o a través de una docker:

```
mkdir viu-crypto-pki
docker run --rm -ti -v $PWD/viu-crypto-pki:/pki alpine/openssl genrsa -aes256 -out /pki/ca.key.pem 4096
 ```

# Definiciones

Sabemos que para utilizar el cifrado asimétrico, es necesario que las personas tengan un clave privada que no le pueden enviar a nadie, y una clave pública que sí que se puede enviar a todo el mundo.

Y aquí está el problema: ¿cómo confiamos en que una clave pública de Alice que nos ha llegado por Internet realmente es la clave pública de Alice?

La solución que hemos encontrado es que la clave pública de Alice venga firmada digitalmente por alguien en quien todoso confiamos, "el jefe de la tribu", lo que técnicamente llamamos "autoridades de certificación"

Cambiar una autoridad de certificación (igual que cambiar al jefe de la tribu) es un proceso costoso. Así que hay que protegerla muy bien para que no sea víctima de ataques informáticos. Y la mejor manera de protegerla es que la Autoridad de Certificación no tenga acceso directo a Internet. Así que en realidad trabajamos en dos pasos:

1. entidad certificador raíz (Root CA) tiene que estar protegida físicamente y aislada de Internet. Solo firma certificados de autoridades de certificación intermedias
2. una autoridad de certificación intermedia sí que firma certificados de los usaurios finales

Así que el certificado de Alice incluye:

1. La identidad de Alice (su nombre, dirección de correo, URL web...)
2. La clave pública de Alice
3. Todo lo anterior está firmado digitalmente por la autoridad de certificación intermedia

De acuerdo, pero para verificar la firma digital de la autoridad de certificación intermedia, necesitamos el certificado de la autoridad de certificación intermedia, que incluye a su vez:

1. La identidad de la autoridad de certificación intermedia
2. La clave pública de la autoridad de certificación intermedia
3. Todo lo anterior está firmado digitalmente por la autoridad de certificación raíz

Y finalmente, para validar la firma de la autoridad de certificación raíz, necesitamos la firma de la autoridad de certificación raíz. Esta vez no hay nadie por encima, así que la autoridad de certificación raíz se firma a sí misma (certificado autofirmado) y confiamos en este certificado porque confiamos en la autoridad de certificación raíz, está instalada en nuestros dispositivos sin más. El certificado de la autoridad de certificación raíz incluye:

1. La identidad de la autoridad de certificación raíz
2. La clave pública de la autoridad de certificación raíz
3. Todo lo anterior está firmado digitalmente por la autoridad de certificación raíz (autofirmado)

Chain_of_trust.svg

En ocasiones, encontraréis páginas web con errores de seguridad "certificado autofirmado". Esto es porque han presentado un certificado que incluye una clave pública firmada por esa misma clave pública y no por una autoridad de certificación raíz en la que todos confiamos. "Esta es mi clave pública porque lo digo yo". No deberíamos confiar en estos certificados, pero se usan en sistemas de prueba o, en ocasiones, para ahorrar dinero (que te firmen un certificado puede ser costar dinero)

Vamos a seguir de firma siplificada los pasos descritos en: https://jamielinux.com/docs/openssl-certificate-authority/index.html

## Creación de la Autoridad Certificadora raíz


Empezamos creando la autoridad de certificación raíz `root-ca`.

Vamos a crear dos directorios: en `root-ca/private` están los archivos privados que nunca deberían salir de la CA, mientras que en `root-ca/certs` están los certificados que pueden hacerse públicos, normalmente instalados en los sistemas operativos.

```
mkdir -p root-ca/private
mkdir -p root-ca/certs
```

Generamos una clave pública/privada para la CA raíz, usando RSA de 4096 bits y protegida con AES 256 (pedirá una contraseña). El archivo de salida lo guardamos en `root-ca/private/ca.key`. Aunque solo sea un archivo, incluye las dos claves, la pública y la privada.

```
openssl genrsa -aes256 -out root-ca/private/root-ca.key 4096
```

Generamos un certificado público para la CA raíz. Este certificado está autofirmado. El archivo de salida lo guardamos en `root-ca/certs/root-ca.cert`. Este es el archivo que debemos convencer a los fabricantes de sistemas operativos para que lo instalen con el sistema operativo.

Pedirá información de la CA, como nombre y localización. Puedes poner lo que mejor te parezca. La contraseña que pide es la de acceso a la clave privada que creamos en el paso anterior.

```
openssl req -key root-ca/private/root-ca.key -new -x509 -days 7300 -sha256 -out root-ca/certs/root-ca.cert
```

Muestra el certificado por pantalla. Comprueba que incluye la identidad de la CA y su clave pública, pero no la clave privada. Está firmada por ella misma: autofirmado.

```
openssl x509 -noout -text -in root-ca/certs/root-ca.cert
```

## Creación de la Autoridad Certificadora intermedia

Vamos a crear una entidad de certificación intermedia que se llame "inter-ca". En la realidad, esta autoridad de certificación está en un dispositivo diferente que la raíz, y puede estar conectada a internet. En este ejercicio simplemente vamos a meterla en un directorio diferente.

```
mkdir -p inter-ca/private
mkdir -p inter-ca/certs
```

Generamos una clave pública/privada para la CA intermedia, usando RSA de 4096 bits y protegida con AES 256 (pedirá una contraseña). El archivo de salida lo guardamos en `inter-ca/private/ca.key`

```
openssl genrsa -aes256 -out inter-ca/private/inter-ca.key 4096
```

Generamos una petición de firmado. Una petición de firmado no es un certificado porque no está firmado, pero sí que incluye toda la información que incluye uncertificado sin la firma. La salida está en `inter-ca/private/inter-ca.csr`. Pedirá una contraseña, ponla diferente que la contraseña de la autoridad raíz para distinguir cuando pide cada una.

```
openssl req -key inter-ca/private/inter-ca.key -new -sha256 -out inter-ca/private/inter-ca.csr
```

Mostamos la petición de firmado por pantalla:

```
openssl req -in inter-ca/private/inter-ca.csr -noout -text
```

Bien, ahora, encendemos el servidor donde esté la autoridad de certificación raíz, le enviamos el archivo CSR de alguna manera (pendrive, probablemente) y obtenemos el certificado intermedio firmado. Eso solo deberíamos hacerlo cada mucho tiempo, dos años por ejemplo, salvo emergencias.

La salida se guarda en `inter-ca/certs/inter-ca.cert`, y la mostramos también por pantalla.

```
openssl x509 -signkey root-ca/private/root-ca.key -in inter-ca/private/inter-ca.csr -req -days 365 -out inter-ca/certs/inter-ca.cert
openssl x509 -noout -text -in inter-ca/certs/inter-ca.cert
```

Fíjate que solo hemos dado una validez de 365 días al nuevo certificado.

Ahora deberíamos desconectar la autoridad raíz y ya solo trabajaos con la intermedia, hasta que necesitemos renovar el certificado de la intermedia


## Firma del certificado de la web www.example.com

Imaginemos ahora que la web www.example.com necesita un certificado. Para eso, tendrá que crear un par de claves público/privada, una petición de firma que le envía a la CA intermedia, y obtiene de la CA intermedia el certificador firmado. El preces es similar al caso anterior.

```
mkdir -p www.example.com/private
mkdir -p www.example.com/certs
```

Generamos una clave pública/privada para www.example.com, usando RSA de 4096 bits y protegida con AES 256 (pedirá una contraseña). El archivo de salida lo guardamos en `www.example.com/private/www.example.com.key`

```
openssl genrsa -aes256 -out www.example.com/private/www.example.com.key 4096
```

Generamos una petición de firmado. Una petición de firmado no es un certificado porque no está firmado, pero sí que incluye toda la información que incluye un certificado sin la firma. La salida está en `www.example.com/private/www.example.com.csr`. Pedirá una contraseña, usa una diferente a la contraseña de las demás para saber cuándo estás usando este certificado. Cuando pida el FQDN, escribe "www.example.com"

```
openssl req -key www.example.com/private/www.example.com.key -new -sha256 -out www.example.com/private/www.example.com.csr
```

Mostamos la petición de firmado por pantalla:

```
openssl req -in www.example.com/private/www.example.com.csr -noout -text
```

Ahora enviamos la petición de firma al servidor donde esté la autoridad intermedia. Este servidor está probablemente en Internet y nos pedirá que demostremos nuestra identidad antes de firmar nada. Cuando esté convencido, hace la firma. La salida se guarda en `www.example.com/certs/www.example.com.cert`

```
openssl x509 -signkey inter-ca/private/inter-ca.key -in www.example.com/private/www.example.com.csr -req -days 365 -out www.example.com/certs/www.example.com.cert
openssl x509 -noout -text -in www.example.com/certs/www.example.com.cert

```

Y el servidor ya tiene un certificado firmado por interm-ca, que a su vez tiene un certificador firmado por root-ca.

Ahora, si tienes un servidor web, tendrías que configurarlo para que usase los archivos `www.example.com/private/www.example.com.key` como clave privada, y `www.example.com/certs/www.example.com.cert` como certificado: https://httpd.apache.org/docs/2.4/ssl/ssl_howto.html